# Install notebook scoped libraries at runtime


#### Topics covered in this example
* Installing notebook scoped libraries at runtime using `install_pypi_package`.
* Converting spark dataFrame to pandas dataFrame.
* `%%display` magic example.

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* The EMR cluster attached to this notebook should have the `Spark` application installed.
* This example uses a public dataset, hence the EMR cluster attached to this notebook must have internet connectivity.
* This notebook uses the `PySpark` kernel.
***

## Introduction
This example shows how to use the notebook-scoped libraries feature of EMR Notebooks to import and install your favorite Python libraries at runtime on your EMR cluster, and use these libraries to enhance your data analysis and visualize your results in rich graphical plots.

The example also shows how to use the `%%display` magic command and how to convert a spark dataFrame to a pandas dataFrame.

The blogpost: <a href="https://aws.amazon.com/blogs/big-data/install-python-libraries-on-a-running-cluster-with-emr-notebooks/" target="_blank">Install Python libraries on a running cluster with EMR Notebooks</a> and document: <a href="https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-managed-notebooks-installing-libraries-and-kernels.html#emr-managed-notebooks-cluster-kernel" target="_blank">Installing and Using Kernels and Libraries</a> provide detailed information.

We use the <a href="https://s3.amazonaws.com/amazon-reviews-pds/readme.html" target="_blank">Amazon customer review dataset</a> that is publically accessible in s3.  
This dataset is a collection of reviews written in the Amazon.com marketplace and associated metadata from 1995 until 2015. This is intended to facilitate study into the properties (and the evolution) of customer reviews potentially including how people evaluate and express their experiences with respect to products at scale.

****

## Example

Examine the current notebook session configuration.

In [ ]:
%%info

Before starting the analysis, check the libraries that are already available on the cluster.  
Use `list_packages()` PySpark API, which lists all the Python libraries on the cluster.

In [ ]:
sc.list_packages()

Load the Amazon customer reviews data for books into a Spark DataFrame.

In [ ]:
df = spark.read.parquet("s3://amazon-reviews-pds/parquet/product_category=Books/*.parquet")

Determine the schema and number of available columns in the dataset.

In [ ]:
# Total columns
print(f"Total Columns: {len(df.dtypes)}")
df.printSchema()

Determine the number of available rows in the dataset.

In [ ]:
# Total row
print(f"Total Rows: {df.count():,}")

Check the total number of books.

In [ ]:
# Total number of books
num_of_books = df.select("product_id").distinct().count()
print(f"Number of Books: {num_of_books:,}")

Import the Pandas library version 0.25.1 and the latest Matplotlib library from the public PyPI repository.  
Install them on the cluster attached to the notebook using the `install_pypi_package` API.

The `install_pypi_package` PySpark API installs the libraries along with any associated dependencies.  
By default, it installs the latest version of the library that is compatible with the Python version you are using.  
A specific version of the library can be installed by specifying the library version from the below Pandas example.

In [ ]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

Verify that the imported packages are successfully installed.

In [ ]:
sc.list_packages()

`%%display` magic command is used to display a spark dataFrame as a beautiful HTML table with horizontal and vertical scroll bar.  
Use the `%%display` magic command to show the number of reviews provided across multiple years.

In [ ]:
%%display
df.groupBy("year").count().orderBy("year")

Analyze the trend for the number of reviews provided across multiple years.   
Use `toPandas()` to convert the Spark dataFrame to a Pandas dataFrame and visualize with Matplotlib.

In [ ]:
# Number of reviews across years
num_of_reviews_by_year = df.groupBy("year").count().orderBy("year").toPandas()

Analyze the number of book reviews by year and find the distribution of customer ratings.

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
num_of_reviews_by_year.plot(kind="area", x="year",y="count", rot=70, color="#bc5090", legend=None, figsize=(8,6))
plt.xticks(num_of_reviews_by_year.year)
plt.xlim(1995, 2015)
plt.title("Number of reviews across years")
plt.xlabel("Year")
plt.ylabel("Number of Reviews")

The preceding commands render the plot on the attached EMR cluster. To visualize the plot within your notebook, use `%matplot` magic.

In [ ]:
%matplot plt

Analyze the distribution of star ratings and visualize it using a pie chart.

In [ ]:
# Distribution of overall star ratings
product_ratings_dist = df.groupBy("star_rating").count().orderBy("count").toPandas()

In [ ]:
plt.clf()
labels = [f"Star Rating: {rating}" for rating in product_ratings_dist["star_rating"]]
reviews = [num_reviews for num_reviews in product_ratings_dist["count"]]
colors = ["#00876c", "#89c079", "#fff392", "#fc9e5a", "#de425b"]
fig, ax = plt.subplots(figsize=(8,5))
w,a,b = ax.pie(reviews, autopct="%1.1f%%", colors=colors)
plt.title("Distribution of star ratings for books")
ax.legend(w, labels, title="Star Ratings", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

The pie chart shows that 80% of users gave a rating of 4 or higher. Approximately 10% of users rated their books 2 or lower. In general, customers are happy about their book purchases from Amazon.

***
## Cleanup

Lastly, use the `uninstall_package` Pyspark API to uninstall the `pandas` and `matplotlib` libraries that were installed using the `install_package` API.

In [ ]:
sc.uninstall_package("pandas")
sc.uninstall_package("matplotlib")

In [ ]:
sc.list_packages()

***
## Additional Notes
To contribute to this example, please contact the EMR Notebooks team.